In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

/home/luke/.virtualenvs/tf1.13_py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/luke/.virtualenvs/tf1.13_py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/luke/.virtualenvs/tf1.13_py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/luke/.virtualenvs/tf1.13_py3/lib/python3

['data_batch_1', 'data_batch_5', 'data_batch_2', 'test_batch', 'batches.meta', 'readme.html', 'data_batch_4', 'data_batch_3']


In [2]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']


class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
# 下面方法是构造分组卷积块
def inception_block(x,
                    output_channel_for_each_path,
                    name):
    """inception block implementation"""
    """
    Args:
    - x:
    - output_channel_for_each_path: eg: [10, 20, 5]，每一个分组卷积的输出通道数
    - name:
    """
    with tf.variable_scope(name):  # 使用scope可以防止命名冲突
        conv1_1 = tf.layers.conv2d(x,
                                   output_channel_for_each_path[0],
                                   (1, 1),
                                   strides = (1,1),
                                   padding = 'same',
                                   activation = tf.nn.relu,
                                   name = 'conv1_1')
        conv3_3 = tf.layers.conv2d(x,
                                   output_channel_for_each_path[1],
                                   (3, 3),
                                   strides = (1,1),
                                   padding = 'same',
                                   activation = tf.nn.relu,
                                   name = 'conv3_3')
        conv5_5 = tf.layers.conv2d(x,
                                   output_channel_for_each_path[2],
                                   (5, 5),
                                   strides = (1,1),
                                   padding = 'same',
                                   activation = tf.nn.relu,
                                   name = 'conv5_5')
        max_pooling = tf.layers.max_pooling2d(x,
                                              (2, 2),
                                              (2, 2),
                                              name = 'max_pooling')
    
    max_pooling_shape = max_pooling.get_shape().as_list()[1:]  # 因为0轴是样本，所以1后面取出的是输出大小
    input_shape = x.get_shape().as_list()[1:]  # 输入大小
    width_padding = (input_shape[0] - max_pooling_shape[0]) // 2  # padding的size参考文档中的表格图
    height_padding = (input_shape[1] - max_pooling_shape[1]) // 2
    # 宽度和高度都做了pad,因为max_pooling后变为一半，补上才能够做concat
    padded_pooling = tf.pad(max_pooling,
                            [[0, 0],
                             [width_padding, width_padding],
                             [height_padding, height_padding],
                             [0, 0]])  
    concat_layer = tf.concat(
        [conv1_1, conv3_3, conv5_5, padded_pooling],
        axis = 3)  # 在通道数维度进行合并
    return concat_layer

# 下面开始搭建inception_net
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
# [None], eg: [0,5,6,3]
x_image = tf.reshape(x, [-1, 3, 32, 32])
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# conv1: 神经元图， feature_map, 输出图像
conv1 = tf.layers.conv2d(x_image,
                         32,  # output channel number
                         (3,3),  # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv1')
pooling1 = tf.layers.max_pooling2d(conv1,
                                   (2, 2),  # kernel size
                                   (2, 2),  # stride
                                   name = 'pool1')
# inception_2a输出通道数是80，图像尺寸是16*16
inception_2a = inception_block(pooling1,
                               [16, 16, 16],
                               name = 'inception_2a')  # 上面封装好的3层卷积加一层池化
# inception_2b 输出通道数是128
inception_2b = inception_block(inception_2a,
                               [16, 16, 16],
                               name = 'inception_2b')
# 图像尺寸是8*8
pooling2 = tf.layers.max_pooling2d(inception_2b,
                                   (2, 2),  # kernel size
                                   (2, 2),  # stride
                                   name = 'pool2')
# inception_3a 输出通道数是176
inception_3a = inception_block(pooling2,
                               [16, 16, 16],
                               name = 'inception_3a')
# inception_3b 输出通道数是224
inception_3b = inception_block(inception_3a,
                               [16, 16, 16],
                               name = 'inception_3b')
# 图像尺寸是4*4
pooling3 = tf.layers.max_pooling2d(inception_3b,
                                   (2, 2),  # kernel size
                                   (2, 2),  # stride
                                   name = 'pool3')

# 展平结果是 4*4*224  3584
flatten = tf.layers.flatten(pooling3)
# print(flatten)  用来验证上面的计算是否正确
y_ = tf.layers.dense(flatten, 10)

# 下面这个和前面是没有区别的
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [ ]:
# 下面和之前是没有区别的
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

# train 10k: 74.65%
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
        if (i+1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print ('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))           

[Train] Step: 100, loss: 1.84884, acc: 0.30000
[Train] Step: 200, loss: 1.45536, acc: 0.45000
[Train] Step: 300, loss: 1.81860, acc: 0.35000
[Train] Step: 400, loss: 1.78338, acc: 0.35000
[Train] Step: 500, loss: 1.61142, acc: 0.45000
[Train] Step: 600, loss: 1.38682, acc: 0.50000
[Train] Step: 700, loss: 1.99419, acc: 0.35000
[Train] Step: 800, loss: 1.56837, acc: 0.50000
[Train] Step: 900, loss: 1.03588, acc: 0.55000
[Train] Step: 1000, loss: 1.62441, acc: 0.45000
(10000, 3072)
(10000,)
[Test ] Step: 1000, acc: 0.53600
[Train] Step: 1100, loss: 0.82034, acc: 0.70000
[Train] Step: 1200, loss: 0.99116, acc: 0.60000
[Train] Step: 1300, loss: 1.26397, acc: 0.55000
[Train] Step: 1400, loss: 0.93753, acc: 0.60000
[Train] Step: 1500, loss: 1.49632, acc: 0.45000
[Train] Step: 1600, loss: 1.20351, acc: 0.60000
[Train] Step: 1700, loss: 0.67753, acc: 0.90000
[Train] Step: 1800, loss: 1.37784, acc: 0.55000
[Train] Step: 1900, loss: 1.33579, acc: 0.50000
[Train] Step: 2000, loss: 1.18075, acc: 0